In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
from snorkel.parser import XMLMultiDocParser

xml_parser = XMLMultiDocParser(
    path='data/Epilepsy/epilepsy_train.xml',
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()')

In [ ]:
from epilepsy_utils import Tagger
from epilepsy_utils import CustomSentenceParser

dg_tagger = Tagger()
sent_parser = CustomSentenceParser(fn=dg_tagger.tag)

In [ ]:
from snorkel.parser import CorpusParser

cp = CorpusParser(xml_parser, sent_parser, max_docs=500)
%time corpus = cp.parse_corpus(name='CDR Train2', session=session)

In [ ]:
session.add(corpus)
session.commit()